In [18]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5 import QtWidgets
from PyQt5.QtCore import *
from PyQt5 import QtCore
from PyQt5.QtWidgets import QMainWindow
from PyQt5.QtWidgets import QApplication
import sys
from os import path
from PyQt5.uic import loadUiType
import requests
from datetime import datetime

In [19]:
# Load the UI file
FORM_CLASS, _ = loadUiType("moodJournaling.ui")

API_URL = "http://localhost:8080/journals/"

In [20]:
class Main(QMainWindow, FORM_CLASS):
    def __init__(self, parent=None):
        super(Main, self).__init__(parent)
        QMainWindow.__init__(self)
        self.setupUi(self)
        self.Handle_Buttons()
        
        # Initialize UI elements (add these if they're not auto-created by loadUiType)
        # These lines are typically not needed if your .ui file properly defines the elements
        # self.moodinput = self.findChild(QLineEdit, "moodinput")
        # self.journalinput = self.findChild(QTextEdit, "journalinput")
        # self.logBtn = self.findChild(QPushButton, "logBtn")
        
    def Handle_Buttons(self):
        # Connect the button click - no need to specify QPushButton in imports
        self.logBtn.clicked.connect(self.log_entry)
        
    def log_entry(self):
        # Get values from the correct widget names
        mood = self.moodinput.text()  # Using text() for QLineEdit
        entry = self.journalinput.toPlainText()  # Using toPlainText() for QTextEdit
        date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        if not entry or not mood:
            QtWidgets.QMessageBox.warning(self, "Warning", "Please enter both mood and journal entry!")
            return
            
        try:
            journal_data = {
                "mood": mood,
                "entry": entry,
                "date": date
            }
            
            response = requests.post(API_URL, json=journal_data)
            
            if response.status_code == 200:
                QtWidgets.QMessageBox.information(self, "Success", "Journal entry saved successfully!")
                self.journalinput.clear()
                self.moodinput.clear()
            else:
                QtWidgets.QMessageBox.critical(self, "Error", 
                    f"Failed to save entry. Status: {response.status_code}\n{response.text}")
                
        except Exception as e:
            QtWidgets.QMessageBox.critical(self, "Error", f"An error occurred: {str(e)}")

In [21]:
def main():
    app = QApplication(sys.argv)
    window = Main()
    window.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

SystemExit: 0